In [ ]:
# Jupyter notebook for testing deepModel library
# author: Fabrizio Romanelli
# email : fabrizio.romanelli@gmail.com
# date  : 04/10/2023

# Import the deepModel library
import deepModel as dm

# Initialize the environment
dm.initialize(CPU=20, GPU=1, VERBOSE='2', NPARRAYS=True)

In [ ]:
# Multivariate DNN dev test

import tensorflow as tf
from keras import backend as K
from keras.losses import MeanSquaredError

def mse_pearson_loss(y_true, y_pred):
  x = y_true
  y = y_pred
  mx = K.mean(x)
  my = K.mean(y)
  xm, ym = x-mx, y-my
  r_num = K.sum(tf.multiply(xm,ym))
  r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
  r = r_num / r_den
  r = K.maximum(K.minimum(r, 1.0), -1.0)
  r = K.square(r)
  mse = MeanSquaredError()
  return 1 - r + mse(y_true, y_pred)

mDnn = dm.multivariateDNN(name="multivariate DNN test", inputN=2)
mDnn.setInputs([{'shape': (2,), 'name': 'gino'}, {'shape': (2,), 'name': 'pino'}])

innerLayers = [[{'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 3, 'activation': 'linear'}]]
innerLayers.append([{'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 3, 'activation': 'linear'}])
innerLayers.append([{'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 16, 'activation': 'elu'}, {'units': 3, 'activation': 'linear'}])

mDnn.setLayers(innerLayers)

outputLayers = [{'units': 32, 'activation': 'elu'}, {'units': 64, 'activation': 'elu'}, {'units': 128, 'activation': 'elu'}, {'units': 64, 'activation': 'elu'}, {'units': 32, 'activation': 'elu'}, {'units': 1, 'activation': 'linear'}]

mDnn.setOutLayers(outputLayers)
mDnn.setModelConfiguration(optimizer='adam', loss='mse')
# mDnn.setModelConfiguration(optimizer='adam', loss=mse_pearson_loss)

mDnn.build()
# mDnn.summary()

import numpy as np

x1 = np.array([0.2,0.3,0.5,0.6,0.7])
x2 = np.array([1.1,1.2,1.4,1.5,1.6])
X1 = np.array([x1,x2]).T
x3 = np.array([0.2,0.3,0.5,0.6,0.7])
x4 = np.array([1.1,1.2,1.4,1.5,1.6])
X2 = np.array([x3,x4]).T
y  = np.array([3.0,4.0,6.0,7.0,8.0])

mDnn.fit(x=[X1,X2], y=y, epochs=50, shuffle=True, verbose=1)

In [ ]:
x1 = np.array([2.4])
x2 = np.array([3.3])
X1 = np.array([x1,x2]).T
x3 = np.array([2.4])
x4 = np.array([3.3])
X2 = np.array([x3,x4]).T
y = mDnn.predict(x=[X1,X2])
print(y)

In [ ]:
mDnn.save('test',tflite=False)

In [ ]:
mDnn2 = dm.multivariateDNN(name="multivariate DNN test 2",inputN=2)
mDnn2.load("test")

In [ ]:
x1 = np.array([2.4])
x2 = np.array([3.3])
X1 = np.array([x1,x2]).T
x3 = np.array([2.4])
x4 = np.array([3.3])
X2 = np.array([x3,x4]).T
y = mDnn2.predict(x=[X1,X2])
print(y)